In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!cp /content/drive/My\ Drive/Colab\ Notebooks/3-class.csv 3-class.csv

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_original = pd.read_csv('3-class.csv')
df_original.head()

,Tweet,Polarity
0,ابراهيم_عيسى الوسخ ابن الوسخه كلما حصل حادث ا...,neg
1,اخطر حروب الارض حرب العقيده حسيبك الله ي اول ...,neg
2,اصبحت تقدم برامج عبر الجمعيات الخيريه لايصال ...,neg
3,اعلامنا متمثل في داوودالشريان و روتانا وطقتهم...,neg
4,الاصرار مرتزقه_برنامج_الاصرار بضاعه هالمترديه...,neg


In [6]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56674 entries, 0 to 56673
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Tweet     56674 non-null  object
 1   Polarity  56674 non-null  object
dtypes: object(2)
memory usage: 885.7+ KB


In [7]:
df_original.Polarity.value_counts()

neg     20731
neut    18726
pos     17217
Name: Polarity, dtype: int64

In [8]:
df_original['Polarity'] = df_original['Polarity'].map({'neg': 0, 'pos': 1, 'neut': 2})

In [9]:
repeat=[]
list_of_tweets = df_original['Tweet'].to_list()
for i in range(len(list_of_tweets)-1):
  if list_of_tweets[i] in list_of_tweets[i+1:]:
    repeat.append(i)
len(repeat)

1498

In [10]:
df_original.drop(repeat,axis=0,inplace=True)
len(df_original)

55176

In [11]:
df_original.reset_index(drop=True, inplace=True)
df_original.head()

,Tweet,Polarity
0,ابراهيم_عيسى الوسخ ابن الوسخه كلما حصل حادث ا...,0
1,اخطر حروب الارض حرب العقيده حسيبك الله ي اول ...,0
2,اصبحت تقدم برامج عبر الجمعيات الخيريه لايصال ...,0
3,اعلامنا متمثل في داوودالشريان و روتانا وطقتهم...,0
4,الاصرار مرتزقه_برنامج_الاصرار بضاعه هالمترديه...,0


In [12]:
df_original['pre_clean_len'] = [len(t) for t in df_original.Tweet]

In [13]:
from pprint import pprint
data_dict = {
    'polarity': {
        'type': df_original.Polarity.dtype,
        'description': 'sentiment class - 0: negative, 1: positive, 2: neutral'
    },
    'tweet': {
        'type': df_original.Tweet.dtype,
        'description': 'tweet text'
    },
    'pre_clean_len': {
        'type': df_original.pre_clean_len.dtype,
        'description': 'length of the tweet before cleaning'
   },
    'dataset.shape': df_original.shape
}

pprint(data_dict)

{'dataset.shape': (55176, 3),
 'polarity': {'description': 'sentiment class - 0: negative, 1: positive, 2: '
                             'neutral',
              'type': dtype('int64')},
 'pre_clean_len': {'description': 'length of the tweet before cleaning',
                   'type': dtype('int64')},
 'tweet': {'description': 'tweet text', 'type': dtype('O')}}


In [14]:
# Data cleaning function definition

from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

In [16]:
import re
from bs4 import BeautifulSoup

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
arabic_num_pat = '[٠١٢٣٤٥٦٧٨٩]'
eng_num_pat = '[0123456789]'
sharta_pat = '[_]'
#sharta_pat = '_[أ-ي]+' #for removing hashtag
eng_pat = '[A-Za-z]'

def tweet_cleaner(text):
  soup = BeautifulSoup(text, 'lxml')
  souped = soup.get_text()
  try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
  except:
        bom_removed = souped
  stripped = re.sub(combined_pat, '', bom_removed)
  stripped = re.sub(www_pat, '', stripped)
  stripped = re.sub(arabic_num_pat, '', stripped)
  stripped = re.sub(eng_num_pat, '', stripped)
  #stripped = re.sub(r'\w+_\s?','_',stripped) #for removing hashtag
  stripped = re.sub(sharta_pat, ' ', stripped)
  stripped = re.sub(eng_pat, ' ', stripped)

  words = [x for x in tok.tokenize(stripped) if len(x) > 1]
  return (" ".join(words)).strip()

In [17]:
clean_tweet_texts = []
for i in range(0, len(df_original)):
  if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed"%(i+1,len(df_original)))                                                                    
  clean_tweet_texts.append(tweet_cleaner(df_original['Tweet'][i]))

Tweets 10000 of 55176 has been processed
Tweets 20000 of 55176 has been processed
Tweets 30000 of 55176 has been processed
Tweets 40000 of 55176 has been processed
Tweets 50000 of 55176 has been processed


In [18]:
#normalization
def normalizeArabic(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "و", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    return(text)
for i in range(len(clean_tweet_texts)):
  clean_tweet_texts[i] = normalizeArabic(clean_tweet_texts[i])

In [19]:
clean_df = pd.DataFrame(clean_tweet_texts, columns=['text'])
clean_df['target'] = df_original.Polarity
clean_df.to_csv('/content/drive/My Drive/clean_tweet.csv', encoding='utf-8')

In [20]:
!cp /content/drive/My\ Drive/clean_tweet.csv /content/

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlinBackend.figure_format = 'retina'

In [22]:
csv = '/content/drive/My Drive/clean_tweet.csv'
df_clean = pd.read_csv(csv, index_col=0)
df_clean.head()

,text,target
0,ابراهيم عيسي الوسخ ابن الوسخه كلما حصل حادث ات...,0
1,اخطر حروب الارض حرب العقيده حسيبك الله اول ال ...,0
2,اصبحت تقدم برامج عبر الجمعيات الخيريه لايصال خ...,0
3,اعلامنا متمثل في داوودالشريان روتانا وطقتهم كي...,0
4,الاصرار مرتزقه برنامج الاصرار بضاعه هالمترديه ...,0


In [23]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55176 entries, 0 to 55175
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    55175 non-null  object
 1   target  55176 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


In [24]:
text=df_clean['text'].copy()
target = df_clean['target'].copy()
my_df= pd.DataFrame(text,columns=['text'])
my_df['target'] = target
my_df.head()

,text,target
0,ابراهيم عيسي الوسخ ابن الوسخه كلما حصل حادث ات...,0
1,اخطر حروب الارض حرب العقيده حسيبك الله اول ال ...,0
2,اصبحت تقدم برامج عبر الجمعيات الخيريه لايصال خ...,0
3,اعلامنا متمثل في داوودالشريان روتانا وطقتهم كي...,0
4,الاصرار مرتزقه برنامج الاصرار بضاعه هالمترديه ...,0


In [25]:
my_df.dropna(inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55175 entries, 0 to 55175
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    55175 non-null  object
 1   target  55175 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


In [26]:
x = my_df.text
y = my_df.target

In [27]:
x = my_df.text
y = my_df.target

In [32]:
from sklearn.model_selection import train_test_split
SEED = 666
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.2, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)


In [33]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
print("Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive, {3:.2f}% neutral".format(len(x_train),
                                                                             (len(x_train[y_train == 0]) / (len(x_train)*1.))*100,
                                                                            (len(x_train[y_train == 1]) / (len(x_train)*1.))*100,
                                                                            (len(x_train[y_train == 2]) / (len(x_train)*1.))*100))
print("Validation set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive, {3:.2f}% neutral".format(len(x_validation),
                                                                             (len(x_validation[y_validation == 0]) / (len(x_validation)*1.))*100,
                                                                            (len(x_validation[y_validation == 1]) / (len(x_validation)*1.))*100,
                                                                            (len(x_validation[y_validation == 2]) / (len(x_validation)*1.))*100))
print("Test set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive, {3:.2f}% neutral".format(len(x_test),
                                                                             (len(x_test[y_test == 0]) / (len(x_test)*1.))*100,
                                                                            (len(x_test[y_test == 1]) / (len(x_test)*1.))*100,
                                                                            (len(x_test[y_test == 2]) / (len(x_test)*1.))*100))


Train set has total 44140 entries with 36.14% negative, 30.83% positive, 33.03% neutral
Validation set has total 5517 entries with 36.52% negative, 30.40% positive, 33.08% neutral
Test set has total 5518 entries with 35.83% negative, 30.10% positive, 34.07% neutral


In [35]:
%%time

param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']} 
clf = Pipeline([('tfidf', TfidfVectorizer()),
                ('grid',GridSearchCV(SVC(),param_grid,refit=True,verbose=3)),
    
])
clf.fit(x_validation,y_validation)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.399, total=   5.4s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.4s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.398, total=   5.5s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   10.9s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.402, total=   5.5s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.390, total=   5.5s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.393, total=   5.5s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.365, total=   5.4s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.365, total=   5.5s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.365, total=   5.5s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.365, total=   5.4s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] .

[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed: 11.4min finished


CPU times: user 11min 30s, sys: 682 ms, total: 11min 31s
Wall time: 11min 32s


In [36]:
%%time

best_clf = Pipeline([('tfidf', TfidfVectorizer()),
                ('svm',SVC(C=1000, gamma=1)),
    
])

best_clf.fit(x_train,y_train)

CPU times: user 24min 56s, sys: 619 ms, total: 24min 57s
Wall time: 24min 58s


In [37]:
best_clf.score(x_validation,y_validation)

0.7794090991480878

In [38]:
from sklearn.metrics import accuracy_score, classification_report
pred = best_clf.predict(x_test)
accuracy_score(pred, y_test)

0.7792678506705328

In [39]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.79      0.77      0.78      2034
           1       0.77      0.85      0.81      1505
           2       0.78      0.74      0.76      1979

    accuracy                           0.78      5518
   macro avg       0.78      0.79      0.78      5518
weighted avg       0.78      0.78      0.78      5518

